# 03 • Model & Blending (CV → OOF → Blend → Calib/τ → Submit)

Цель: за 2–3 клика обучить/собрать кандидатов, сравнить по OOF, сделать бленд, при необходимости откалибровать/подобрать порог и сформировать сабмит.

Шаги:
1) Выбери `RUN_TAG` набора фич → «Проверить набор»
2) Отметь кандидатов и запусти `Dry-run`, затем `Train selected`
3) Загрузить готовые прогоны → сравнить → собрать бленд
4) (Опционально) Калибровка/τ
5) Собрать сабмит CSV

In [ ]:
from IPython.display import display, HTML
display(HTML("""
<style>
/* ipywidgets v8 (JupyterLab 4) */
.jp-OutputArea .widget-button .widget-label { 
  white-space: normal !important; 
  overflow: visible !important; 
  text-overflow: clip !important;
  line-height: 1.2 !important;
}
/* fallback для ipywidgets v7 */
.jupyter-widgets.widget-button .widget-label {
  white-space: normal !important; 
  overflow: visible !important; 
  text-overflow: clip !important;
  line-height: 1.2 !important;
}
</style>
"""))


In [ ]:
import os, sys, json, time, math, subprocess, textwrap, shutil, pickle, gc, uuid, warnings, glob
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Tuple, Any

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")
plt.rcParams["figure.figsize"] = (10, 5)

try:
    import ipywidgets as w
    from IPython.display import display, clear_output, HTML
except Exception as e:
    raise RuntimeError("Нужен пакет ipywidgets (pip install ipywidgets)") from e

# -------------------- helpers --------------------
def now_tag(prefix="run"):
    return f"{prefix}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

def ensure_dir(p: Path) -> Path:
    p.mkdir(parents=True, exist_ok=True)
    return p

def run_cmd(args, cwd=None, stream=False, log_file=None):
    """
    args: list[str] - команда
    stream: True → потоковый вывод в ячейку
    log_file: путь для записи лога (append)
    """
    if stream:
        proc = subprocess.Popen(args, cwd=cwd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1)
        lines = []
        if log_file:
            Path(log_file).parent.mkdir(parents=True, exist_ok=True)
            lf = open(log_file, "a", encoding="utf-8")
        else:
            lf = None
        try:
            for line in proc.stdout:
                print(line, end="")
                if lf: lf.write(line)
                lines.append(line)
        finally:
            if lf: lf.close()
        code = proc.wait()
        return code, "".join(lines)
    else:
        res = subprocess.run(args, cwd=cwd, capture_output=True, text=True)
        if log_file:
            Path(log_file).parent.mkdir(parents=True, exist_ok=True)
            with open(log_file, "a", encoding="utf-8") as f:
                f.write(res.stdout)
                f.write(res.stderr)
        return res.returncode, (res.stdout + res.stderr)

def tail_log(path, n=120):
    p = Path(path)
    if not p.exists():
        return f"[нет лога] {p}"
    with p.open("r", encoding="utf-8", errors="ignore") as f:
        data = f.readlines()
    return "".join(data[-n:])

def read_json(path):
    p = Path(path)
    if not p.exists():
        return None
    try:
        return json.loads(p.read_text(encoding="utf-8"))
    except Exception:
        return None

def write_json(path, obj):
    p = Path(path); p.parent.mkdir(parents=True, exist_ok=True)
    p.write_text(json.dumps(obj, ensure_ascii=False, indent=2), encoding="utf-8")

def try_read_parquet(path):
    p = Path(path)
    if not p.exists():
        return None
    try:
        return pd.read_parquet(p)
    except Exception as e:
        try:
            import fastparquet  # noqa
            return pd.read_parquet(p, engine="fastparquet")
        except Exception:
            raise e

def mem_df_gb(df: pd.DataFrame) -> float:
    try:
        return float(df.memory_usage(deep=True).sum())/(1024**3)
    except Exception:
        return 0.0

def list_run_tags(sets_root="artifacts/sets"):
    base = Path(sets_root)
    if not base.exists():
        return []
    return sorted([p.name for p in base.iterdir() if p.is_dir()])

UI_STATE_PATH = Path(".tmp/model_ui_state.json")
def load_ui_state():
    return read_json(UI_STATE_PATH) or {}

def save_ui_state(d):
    write_json(UI_STATE_PATH, d)

def autoload_or(default, key):
    st = load_ui_state()
    return st.get(key, default)

def badge(text, color="#0a0"):
    return HTML(f"<span style='display:inline-block;background:{color};color:#fff;padding:3px 8px;border-radius:7px;font-weight:600'>{text}</span>")

print("Python:", sys.version.split()[0])


In [ ]:
BTN_LAYOUT = w.Layout(min_width="220px", width="auto", height="36px", flex="0 0 auto")
ROW_LAYOUT = w.Layout(flex_flow="row wrap", grid_gap="8px")
GRID_LAYOUT = w.Layout(grid_template_columns="repeat(3, minmax(220px, 1fr))", grid_gap="8px")

In [ ]:
# выбор набора фич
RUN_TAG = w.Combobox(
    placeholder="RUN_TAG (из artifacts/sets/*)",
    options=list_run_tags(),
    value=autoload_or("", "RUN_TAG"),
    description="RUN_TAG",
    ensure_option=False,
    layout=w.Layout(width="50%")
)
SETS_DIR = w.Text(value=autoload_or("", "SETS_DIR"), description="SETS_DIR", layout=w.Layout(width="70%"))
BTN_AUTOFILL = w.Button(description="Автоподстановка", button_style="info", layout=BTN_LAYOUT)
BTN_CHECK_SET = w.Button(description="Проверить набор", button_style="primary", layout=BTN_LAYOUT)
OUT_SET = w.Output(layout={'border':'1px solid #ccc'})

def autofill_sets_dir(_):
    tag = RUN_TAG.value.strip()
    if tag:
        SETS_DIR.value = f"artifacts/sets/{tag}"

def check_set(_):
    OUT_SET.clear_output(wait=True)
    with OUT_SET:
        sd = Path(SETS_DIR.value) if SETS_DIR.value.strip() else (Path("artifacts/sets")/RUN_TAG.value.strip())
        print("SETS_DIR:", sd.as_posix())
        need = ["y_train.parquet","ids_train.parquet","ids_test.parquet","folds.pkl"]
        for f in need:
            print(f, "✓" if (sd/f).exists() else "❌")
        Xdtr, Xdte = sd/"X_dense_train.parquet", sd/"X_dense_test.parquet"
        Xstr, Xste = sd/"X_sparse_train.npz", sd/"X_sparse_test.npz"
        print("Dense:", "OK" if (Xdtr.exists() and Xdte.exists()) else "—",
              "| Sparse:", "OK" if (Xstr.exists() and Xste.exists()) else "—")
        if Xdtr.exists():
            try:
                x = try_read_parquet(Xdtr)
                print("Dense train shape:", x.shape, "| mem:", round(mem_df_gb(x),3),"GB")
            except Exception as e:
                print("dense чтение:", e)
        if (sd/"folds.pkl").exists():
            import pickle
            folds = pickle.loads((sd/"folds.pkl").read_bytes())
            print("Folds:", len(folds), "| вал. размеры:", [len(v) for _,v in folds[:5]], "...")
        if (sd/"meta.json").exists():
            meta = read_json(sd/"meta.json") or {}
            print("META keys:", list(meta.keys()))

BTN_AUTOFILL.on_click(autofill_sets_dir)
BTN_CHECK_SET.on_click(check_set)
display(w.VBox([w.HBox([RUN_TAG, BTN_AUTOFILL], layout=ROW_LAYOUT), SETS_DIR, w.HBox([BTN_CHECK_SET], layout=ROW_LAYOUT), OUT_SET]))


In [ ]:
# автопоиск yaml-кандидатов
def list_candidates(config_dir="configs/models"):
    items = []
    for p in sorted(Path(config_dir).glob("*.yaml")):
        items.append(p.stem)
    return items

CANDS_ALL = list_candidates()

CANDS_MULTI = w.SelectMultiple(
    options=CANDS_ALL, value=tuple(autoload_or([], "CANDS_MULTI") or []),
    description="cands", rows=10, layout=w.Layout(width="45%")
)

TASK  = w.Dropdown(options=["auto","binary","multiclass","regression","multilabel"], value=autoload_or("auto","TASK"), description="TASK")
METRIC= w.Dropdown(options=["roc_auc","pr_auc","logloss","accuracy","f1","rmse","mae","mape"], value=autoload_or("roc_auc","METRIC"), description="METRIC")
DEVICE= w.Dropdown(options=["auto","cpu","gpu"], value=autoload_or("auto","DEVICE"), description="DEVICE")
THREADS = w.IntText(value=int(autoload_or(-1,"THREADS")), description="THREADS")
SEED    = w.IntText(value=int(autoload_or(42,"SEED")), description="SEED")
TIMEOUT = w.IntText(value=int(autoload_or(0,"TIMEOUT")), description="TIMEOUT_MIN")
RESUME  = w.Checkbox(value=bool(autoload_or(True,"RESUME")), description="RESUME")
CALIB   = w.Dropdown(options=["off","platt","isotonic"], value=autoload_or("off","CALIB"), description="CALIBRATE")
SAVE_TEST = w.Checkbox(value=bool(autoload_or(True,"SAVE_TEST")), description="SAVE_TEST")

display(w.HBox([CANDS_MULTI, w.VBox([TASK, METRIC, DEVICE, THREADS, SEED, TIMEOUT, RESUME, CALIB, SAVE_TEST])], layout=ROW_LAYOUT))


In [ ]:
BTN_DRY = w.Button(description="Dry-run (план)", button_style="warning", layout=BTN_LAYOUT)
OUT_DRY = w.Output(layout={'border':'1px dashed #e0a'})

def on_dry(_):
    OUT_DRY.clear_output(wait=True)
    with OUT_DRY:
        tag = RUN_TAG.value.strip()
        sd  = SETS_DIR.value.strip() or f"artifacts/sets/{tag}"
        cands = list(CANDS_MULTI.value)
        if not tag or not cands:
            print("Укажи RUN_TAG и выбери хотя бы одного кандидата.")
            return
        args = [
            sys.executable, "tools/run_model.py",
            "--tag", tag,
            "--cands", ",".join(cands),
            "--metric", METRIC.value,
            "--device", DEVICE.value,
            "--threads", str(THREADS.value),
            "--seed", str(SEED.value),
            "--sets-dir", sd,
            "--dry-run"
        ]
        if TASK.value!="auto": args += ["--task", TASK.value]
        if RESUME.value: args += ["--resume"]
        if TIMEOUT.value>0: args += ["--timeout-min", str(TIMEOUT.value)]
        if SAVE_TEST.value: args += ["--save-test"]
        if CALIB.value!="off": args += ["--calibrate", CALIB.value]
        print("CMD:", " ".join(args))
        code, out = run_cmd(args, stream=False, log_file=None)
        print(out if out.strip() else f"[exit code {code}]")
        print()
        print("Если ок — запускай обучение.")

BTN_DRY.on_click(on_dry)
display(w.VBox([BTN_DRY, OUT_DRY]))


In [ ]:
BTN_TRAIN = w.Button(description="Train selected", button_style="success", layout=BTN_LAYOUT)
OUT_TRAIN = w.Output(layout={'border':'1px solid #ccc', 'height':'300px', 'overflow_y':'auto'})
OUT_SUM   = w.Output(layout={'border':'1px solid #ccc'})

def on_train(_):
    OUT_TRAIN.clear_output(wait=True); OUT_SUM.clear_output(wait=True)
    with OUT_TRAIN:
        tag = RUN_TAG.value.strip()
        sd  = SETS_DIR.value.strip() or f"artifacts/sets/{tag}"
        cands = list(CANDS_MULTI.value)
        if not tag or not cands:
            print("Укажи RUN_TAG и выбери кандидатов.")
            return
        args = [
            sys.executable, "tools/run_model.py",
            "--tag", tag,
            "--cands", ",".join(cands),
            "--metric", METRIC.value,
            "--device", DEVICE.value,
            "--threads", str(THREADS.value),
            "--seed", str(SEED.value),
            "--sets-dir", sd
        ]
        if TASK.value!="auto": args += ["--task", TASK.value]
        if RESUME.value: args += ["--resume"]
        if TIMEOUT.value>0: args += ["--timeout-min", str(TIMEOUT.value)]
        if SAVE_TEST.value: args += ["--save-test"]
        if CALIB.value!="off": args += ["--calibrate", CALIB.value]
        log_file = Path("artifacts")/"models"/f"log_{tag}.txt"
        print("CMD:", " ".join(args))
        print("Лог:", log_file.as_posix())
        code, _ = run_cmd(args, stream=True, log_file=log_file)
        print("[EXIT CODE]", code)
    # сводка по новым моделям
    with OUT_SUM:
        idx_path = Path("artifacts/models/index.json")
        if idx_path.exists():
            idx = read_json(idx_path) or {}
            rows=[]
            for rid, rec in idx.items():
                if rec.get("tag")==tag:
                    rows.append({
                        "run_id": rid,
                        "cand": rec.get("cand"),
                        "lib": rec.get("lib"),
                        "task": rec.get("task"),
                        "metric": rec.get("metric"),
                        "cv_mean": rec.get("cv_mean"),
                        "cv_std": rec.get("cv_std"),
                        "time_sec": rec.get("time_sec"),
                        "path": rec.get("path")
                    })
            if rows:
                df = pd.DataFrame(rows).sort_values("cv_mean", ascending=False)
                display(df)
                # сохраним
                ensure_dir(Path("artifacts")/"models")
                df.to_csv(Path("artifacts")/"models"/f"last_runs_{tag}.csv", index=False)
            else:
                print("Пока нет записей в index.json для этого RUN_TAG.")
        else:
            print("artifacts/models/index.json не найден.")

BTN_TRAIN.on_click(on_train)
display(w.VBox([BTN_TRAIN, OUT_TRAIN, OUT_SUM]))


In [ ]:
BTN_REFRESH_RUNS = w.Button(description="Обновить список", button_style="info", layout=BTN_LAYOUT)
RUNS_MULTI = w.SelectMultiple(options=[], description="run_id", rows=12, layout=w.Layout(width="80%"))
OUT_RUNS   = w.Output()

def refresh_runs(_=None):
    tag = RUN_TAG.value.strip()
    idx_path = Path("artifacts/models/index.json")
    items=[]
    if idx_path.exists():
        idx = read_json(idx_path) or {}
        for rid, rec in idx.items():
            if rec.get("tag")==tag:
                items.append((f"{rec.get('cv_mean'):.6f} | {rec.get('cand')} | {rid}", rid))
    RUNS_MULTI.options = [rid for _,rid in sorted(items, reverse=True)]

BTN_REFRESH_RUNS.on_click(refresh_runs)
refresh_runs()
display(w.HBox([BTN_REFRESH_RUNS], layout=ROW_LAYOUT), RUNS_MULTI, OUT_RUNS)


In [ ]:
BTN_LOAD_PREDS = w.Button(description="Загрузить OOF/test", button_style="primary", layout=BTN_LAYOUT)
OUT_LOAD = w.Output(layout={'border':'1px solid #ccc'})

# храним загруженные предсказания в памяти
LOADED = {"run_ids": [], "oof": {}, "test": {}, "paths": {}}

def norm_preds(task, arr):
    a = np.asarray(arr)
    if task=="binary":
        if a.ndim==2 and a.shape[1]==1: return a.reshape(-1)
        if a.ndim==2 and a.shape[1]==2: return a[:,1]
        if a.ndim==1: return a
        return a.reshape(-1)
    return a  # multiclass/regression
       
def on_load_preds(_):
    OUT_LOAD.clear_output(wait=True)
    with OUT_LOAD:
        ids = list(RUNS_MULTI.value)
        if not ids:
            print("Выбери хотя бы один run_id.")
            return
        idx_path = Path("artifacts/models/index.json")
        idx = read_json(idx_path) or {}
        # определим task
        run0 = idx[ids[0]]
        task = run0.get("task","binary")
        LOADED["run_ids"].clear()
        LOADED["oof"].clear()
        LOADED["test"].clear()
        LOADED["paths"].clear()
        for rid in ids:
            rec = idx.get(rid, {})
            p = Path(rec.get("path",""))
            oof_p = p/"oof.npy"
            test_p = p/"test_pred.npy"
            if not oof_p.exists():
                print("skip:", rid, "— нет oof.npy")
                continue
            oof = np.load(oof_p)
            test = np.load(test_p) if test_p.exists() else None
            LOADED["run_ids"].append(rid)
            LOADED["oof"][rid] = norm_preds(task, oof)
            LOADED["test"][rid] = None if test is None else norm_preds(task, test)
            LOADED["paths"][rid] = p.as_posix()
            print("OK:", rid, "| OOF:", LOADED["oof"][rid].shape, "| TEST:", None if LOADED["test"][rid] is None else LOADED["test"][rid].shape)
        print("Загружено:", len(LOADED["run_ids"]), "кандидатов.")

BTN_LOAD_PREDS.on_click(on_load_preds)
display(BTN_LOAD_PREDS, OUT_LOAD)


In [ ]:
BTN_ANALYZE = w.Button(description="Сводка и корреляции", button_style="", layout=BTN_LAYOUT)
OUT_ANALYZE = w.Output(layout={'border':'1px solid #ccc'})

def metric_fn(task: str, metric: str):
    m = metric.lower()
    if task in ("binary","multiclass","multilabel"):
        from sklearn.metrics import roc_auc_score, average_precision_score, log_loss, accuracy_score, f1_score
        if m in ("roc_auc","auc"):
            def _f(y_true, y_pred):
                if task=="binary":
                    return roc_auc_score(y_true, y_pred.reshape(-1))
                elif task=="multiclass":
                    from sklearn.preprocessing import label_binarize
                    classes = np.unique(y_true)
                    Y = label_binarize(y_true, classes=classes)
                    return roc_auc_score(Y, y_pred, average="macro", multi_class="ovr")
                else:
                    return roc_auc_score(y_true, y_pred, average="macro")
            return _f
        if m in ("pr_auc","ap","average_precision"):
            def _f(y_true, y_pred):
                if task=="binary":
                    return average_precision_score(y_true, y_pred.reshape(-1))
                elif task=="multiclass":
                    from sklearn.preprocessing import label_binarize
                    classes = np.unique(y_true)
                    Y = label_binarize(y_true, classes=classes)
                    return average_precision_score(Y, y_pred, average="macro")
                else:
                    return average_precision_score(y_true, y_pred, average="macro")
            return _f
        if m == "logloss":
            def _f(y_true, y_pred):
                if task=="binary":
                    p = np.clip(y_pred.reshape(-1), 1e-15, 1-1e-15)
                    P = np.vstack([1-p,p]).T
                    return log_loss(y_true, P, labels=[0,1])
                elif task=="multiclass":
                    return log_loss(y_true, y_pred)
                else:
                    raise ValueError
            return _f
        if m in ("accuracy","acc"):
            def _f(y_true, y_pred):
                if task=="binary":
                    return ((y_pred.reshape(-1)>=0.5).astype(int)==y_true).mean()
                elif task=="multiclass":
                    return (np.argmax(y_pred,1)==y_true).mean()
                else:
                    raise ValueError
            return _f
        if m in ("f1","macro_f1"):
            def _f(y_true, y_pred):
                from sklearn.metrics import f1_score
                if task=="binary":
                    return f1_score(y_true, (y_pred.reshape(-1)>=0.5).astype(int))
                elif task=="multiclass":
                    return f1_score(y_true, np.argmax(y_pred,1), average="macro")
                else:
                    raise ValueError
            return _f
    if metric=="rmse":
        from sklearn.metrics import mean_squared_error
        return lambda y_true, y_pred: math.sqrt(mean_squared_error(y_true, y_pred))
    if metric=="mae":
        from sklearn.metrics import mean_absolute_error
        return lambda y_true, y_pred: mean_absolute_error(y_true, y_pred)
    if metric=="mape":
        def _m(y_true, y_pred):
            y_true = np.asarray(y_true, float)
            y_pred = np.asarray(y_pred, float)
            eps = 1e-9
            return np.mean(np.abs((y_true-y_pred)/np.clip(np.abs(y_true), eps, None)))*100.0
        return _m
    raise ValueError("Неизвестная метрика")

def on_analyze(_):
    OUT_ANALYZE.clear_output(wait=True)
    with OUT_ANALYZE:
        tag = RUN_TAG.value.strip()
        sd  = Path(SETS_DIR.value.strip() or f"artifacts/sets/{tag}")
        ydf = try_read_parquet(sd/"y_train.parquet")
        if ydf is None:
            print("Нет y_train.parquet")
            return
        target_col = [c for c in ydf.columns if c!=ydf.columns[0]][0]
        y = ydf[target_col].to_numpy()
        # определим task
        task = TASK.value if TASK.value!="auto" else ("binary" if len(np.unique(y))<=2 else "multiclass")
        # таблица метрик для загруженных
        if not LOADED["run_ids"]:
            print("Сначала загрузите OOF/test (прошлая ячейка).")
            return
        scorer = metric_fn(task, METRIC.value)
        rows=[]
        mats=[]
        names=[]
        for rid in LOADED["run_ids"]:
            oof = LOADED["oof"][rid]
            rows.append({"run_id": rid, "metric": scorer(y, oof)})
            mats.append(oof.reshape(-1,1) if task=="binary" else oof)
            names.append(rid[:10])
        df = pd.DataFrame(rows).sort_values("metric", ascending=False)
        display(df)
        # корреляции OOF
        try:
            if len(mats)>=2:
                M = np.hstack([m if m.ndim==2 and m.shape[1]==1 else m.max(1).reshape(-1,1) for m in mats])  # упрощённая редукция для нелинейности
                C = np.corrcoef(M.T)
                plt.figure()
                plt.imshow(C, interpolation="nearest")
                plt.colorbar(); plt.title("Корреляция OOF (сжатая)")
                plt.xticks(range(len(names)), names, rotation=90)
                plt.yticks(range(len(names)), names)
                plt.tight_layout(); plt.show()
        except Exception as e:
            print("corr skipped:", e)

BTN_ANALYZE.on_click(on_analyze)
display(BTN_ANALYZE, OUT_ANALYZE)


In [ ]:
# выбор на бленд
BLEND_RUNS = w.SelectMultiple(options=[], description="В бленд", rows=10, layout=w.Layout(width="80%"))
BTN_REFRESH_FOR_BLEND = w.Button(description="← из загруженных", button_style="info", layout=BTN_LAYOUT)
BTN_EQ   = w.Button(description="Equal-weight", button_style="success", layout=BTN_LAYOUT)
BTN_DIR  = w.Button(description="Dirichlet-search", button_style="warning", layout=BTN_LAYOUT)
BTN_L2   = w.Button(description="Level-2 (Ridge/LogReg)", button_style="primary", layout=BTN_LAYOUT)
OUT_BLEND= w.Output(layout={'border':'1px solid #ccc'})

def refresh_for_blend(_=None):
    BLEND_RUNS.options = list(LOADED["run_ids"])

BTN_REFRESH_FOR_BLEND.on_click(refresh_for_blend)
display(w.HBox([BTN_REFRESH_FOR_BLEND], layout=ROW_LAYOUT), BLEND_RUNS, w.HBox([BTN_EQ, BTN_DIR, BTN_L2], layout=ROW_LAYOUT), OUT_BLEND)

BLEND_STATE = {"oof": None, "test": None, "weights": None, "type": None, "members": None, "metric": None}

def _collect_stack(task, members):
    ys = []
    tests = []
    for rid in members:
        ys.append(LOADED["oof"][rid].reshape(-1,1) if task=="binary" else LOADED["oof"][rid])
        t = LOADED["test"][rid]
        if t is None:
            raise RuntimeError(f"{rid}: нет test_pred.npy")
        tests.append(t.reshape(-1,1) if task=="binary" else t)
    return ys, tests

def blend_equal(task, y_true, members):
    Ys, Ts = _collect_stack(task, members)
    Y = np.hstack(Ys)
    T = np.hstack(Ts)
    oof_bl = np.mean(Y, axis=1) if task!="multiclass" else np.mean(Y, axis=2)  # multiclass Y stacking shape?
    if task=="multiclass":
        oof_bl = np.mean(np.stack(Ys, axis=0), axis=0)  # (m, n, C) -> (n, C)
        test_bl= np.mean(np.stack(Ts, axis=0), axis=0)
    else:
        oof_bl = np.mean(np.hstack(Ys), axis=1)
        test_bl= np.mean(np.hstack(Ts), axis=1)
    return oof_bl, test_bl, np.ones(len(members))/len(members)

def dirichlet_search(task, y_true, members, metric, n_samples=2000, seed=42):
    rng = np.random.default_rng(seed)
    Ys, Ts = _collect_stack(task, members)
    if task=="multiclass":
        Y = np.stack(Ys, axis=0)  # (m, n, C)
        T = np.stack(Ts, axis=0)  # (m, nt, C)
    else:
        Y = np.hstack(Ys)         # (n, m)
        T = np.hstack(Ts)         # (nt, m)
    scorer = metric_fn(task, metric)
    best_w, best_s = None, -1e9
    for _ in range(n_samples):
        w = rng.dirichlet([1.0]*len(members))
        if task=="multiclass":
            oof = np.tensordot(w, Y, axes=(0,0))  # (n, C)
        else:
            oof = (Y @ w).reshape(-1)
        s = scorer(y_true, oof)
        if s>best_s:
            best_s, best_w = s, w
    if task=="multiclass":
        test_bl = np.tensordot(best_w, T, axes=(0,0))
        oof_bl  = np.tensordot(best_w, Y, axes=(0,0))
    else:
        test_bl = (T @ best_w).reshape(-1)
        oof_bl  = (Y @ best_w).reshape(-1)
    return oof_bl, test_bl, best_w

def level2_stack(task, y_true, members):
    from sklearn.linear_model import Ridge, LogisticRegression
    Ys, Ts = _collect_stack(task, members)
    if task=="binary":
        X = np.hstack(Ys)     # (n, m)
        Xt= np.hstack(Ts)     # (nt, m)
        mdl = LogisticRegression(max_iter=1000)
        mdl.fit(X, y_true.astype(int))
        oof = mdl.predict_proba(X)[:,1]
        test= mdl.predict_proba(Xt)[:,1]
        return oof, test, mdl
    elif task=="multiclass":
        X = np.hstack([y for y in Ys])  # (n, m*C) упрощённо — берём argmax proba? лучше конкатенировать все классы
        Xt= np.hstack([t for t in Ts])
        mdl = Ridge(alpha=1.0)
        mdl.fit(X, pd.get_dummies(y_true).values)
        oof = np.clip(mdl.predict(X), 0, 1)
        oof = oof / np.clip(oof.sum(1, keepdims=True), 1e-9, None)
        test = np.clip(mdl.predict(Xt), 0, 1)
        test = test / np.clip(test.sum(1, keepdims=True), 1e-9, None)
        return oof, test, mdl
    else:  # regression
        X = np.hstack(Ys)
        Xt= np.hstack(Ts)
        mdl = Ridge(alpha=1.0)
        mdl.fit(X, y_true)
        return mdl.predict(X), mdl.predict(Xt), mdl

def on_blend_equal(_):
    OUT_BLEND.clear_output(wait=True)
    with OUT_BLEND:
        tag = RUN_TAG.value.strip()
        sd  = Path(SETS_DIR.value.strip() or f"artifacts/sets/{tag}")
        ydf = try_read_parquet(sd/"y_train.parquet")
        y = ydf[[c for c in ydf.columns if c!=ydf.columns[0]][0]].to_numpy()
        task = TASK.value if TASK.value!="auto" else ("binary" if len(np.unique(y))<=2 else "multiclass")
        members = list(BLEND_RUNS.value)
        if not members: print("Выбери участников бленда"); return
        oof, test, w = blend_equal(task, y, members)
        scorer = metric_fn(task, METRIC.value); s = scorer(y, oof)
        print("Equal-weight blend:", "metric=", s)
        BLEND_STATE.update({"oof":oof, "test":test, "weights":w, "type":"equal", "members":members, "metric":float(s)})

def on_blend_dirichlet(_):
    OUT_BLEND.clear_output(wait=True)
    with OUT_BLEND:
        tag = RUN_TAG.value.strip()
        sd  = Path(SETS_DIR.value.strip() or f"artifacts/sets/{tag}")
        ydf = try_read_parquet(sd/"y_train.parquet")
        y = ydf[[c for c in ydf.columns if c!=ydf.columns[0]][0]].to_numpy()
        task = TASK.value if TASK.value!="auto" else ("binary" if len(np.unique(y))<=2 else "multiclass")
        members = list(BLEND_RUNS.value)
        if not members: print("Выбери участников бленда"); return
        oof, test, w = dirichlet_search(task, y, members, METRIC.value, n_samples=2000, seed=42)
        scorer = metric_fn(task, METRIC.value); s = scorer(y, oof)
        print("Dirichlet-search blend:", "metric=", s, "| weights:", np.round(w,4))
        BLEND_STATE.update({"oof":oof, "test":test, "weights":w, "type":"dirichlet", "members":members, "metric":float(s)})

def on_blend_l2(_):
    OUT_BLEND.clear_output(wait=True)
    with OUT_BLEND:
        tag = RUN_TAG.value.strip()
        sd  = Path(SETS_DIR.value.strip() or f"artifacts/sets/{tag}")
        ydf = try_read_parquet(sd/"y_train.parquet")
        y = ydf[[c for c in ydf.columns if c!=ydf.columns[0]][0]].to_numpy()
        task = TASK.value if TASK.value!="auto" else ("binary" if len(np.unique(y))<=2 else "multiclass")
        members = list(BLEND_RUNS.value)
        if not members: print("Выбери участников бленда"); return
        oof, test, mdl = level2_stack(task, y, members)
        scorer = metric_fn(task, METRIC.value); s = scorer(y, oof)
        print("Level-2 blend:", "metric=", s)
        BLEND_STATE.update({"oof":oof, "test":test, "weights":None, "type":"level2", "members":members, "metric":float(s)})

BTN_EQ.on_click(on_blend_equal)
BTN_DIR.on_click(on_blend_dirichlet)
BTN_L2.on_click(on_blend_l2)


In [ ]:
BTN_SAVE_BLEND = w.Button(description="Сохранить бленд", button_style="success", layout=BTN_LAYOUT)
OUT_SAVE_BLEND = w.Output()

def on_save_blend(_):
    OUT_SAVE_BLEND.clear_output(wait=True)
    with OUT_SAVE_BLEND:
        if BLEND_STATE["oof"] is None or BLEND_STATE["test"] is None:
            print("Сначала соберите бленд.")
            return
        tag = RUN_TAG.value.strip()
        blend_id = f"blend_{BLEND_STATE['type']}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
        out_dir = ensure_dir(Path("artifacts")/"models"/"blends"/blend_id)
        np.save(out_dir/"oof.npy", BLEND_STATE["oof"])
        np.save(out_dir/"test_pred.npy", BLEND_STATE["test"])
        cfg = {
            "run_tag": tag,
            "type": BLEND_STATE["type"],
            "members": BLEND_STATE["members"],
            "weights": None if BLEND_STATE["weights"] is None else list(map(float, BLEND_STATE["weights"])),
            "metric": BLEND_STATE["metric"]
        }
        write_json(out_dir/"blend_config.json", cfg)
        print("blend saved →", out_dir.as_posix())

BTN_SAVE_BLEND.on_click(on_save_blend)
display(BTN_SAVE_BLEND, OUT_SAVE_BLEND)


In [ ]:
CALIB_MODE = w.Dropdown(options=["off","platt","isotonic"], value="off", description="Calibrate")
THR_MODE   = w.Dropdown(options=["f1","youden"], value="f1", description="τ strategy")
BTN_CALIB  = w.Button(description="Калибровать бленд/кандидат", button_style="warning", layout=BTN_LAYOUT)
OUT_CALIB  = w.Output(layout={'border':'1px solid #ccc'})

CALIB_STATE = {"cal": None, "tau": None, "oof_cal": None, "test_cal": None}

def fit_calibrator(method, y, p):
    method = method.lower()
    if method in ("off","none",""): return None, lambda z: z
    if method=="platt":
        from sklearn.linear_model import LogisticRegression
        lr = LogisticRegression(max_iter=1000)
        lr.fit(p.reshape(-1,1), y.astype(int))
        return ("platt", lr), (lambda z: lr.predict_proba(z.reshape(-1,1))[:,1])
    if method=="isotonic":
        from sklearn.isotonic import IsotonicRegression
        ir = IsotonicRegression(out_of_bounds="clip")
        ir.fit(p.reshape(-1), y.astype(int))
        return ("isotonic", ir), (lambda z: ir.predict(z.reshape(-1)))
    raise ValueError

def find_tau(y, p, mode="f1"):
    from sklearn.metrics import f1_score, roc_curve
    if mode=="f1":
        best, best_t = -1, 0.5
        for t in np.linspace(0,1,401):
            s = f1_score(y, (p>=t).astype(int))
            if s>best: best, best_t = s, t
        return best_t, best
    # Youden J on ROC
    fpr, tpr, thr = roc_curve(y, p)
    j = tpr - fpr
    i = int(np.argmax(j))
    return float(thr[i]), float(j[i])

def on_calib(_):
    OUT_CALIB.clear_output(wait=True)
    with OUT_CALIB:
        tag = RUN_TAG.value.strip()
        sd  = Path(SETS_DIR.value.strip() or f"artifacts/sets/{tag}")
        ydf = try_read_parquet(sd/"y_train.parquet")
        y = ydf[[c for c in ydf.columns if c!=ydf.columns[0]][0]].to_numpy()
        task = TASK.value if TASK.value!="auto" else ("binary" if len(np.unique(y))<=2 else "multiclass")
        if task!="binary":
            print("Калибровка/τ активны только для binary.")
            return
        # источник: если есть BLEND_STATE, используем его; иначе — первый загруженный
        if BLEND_STATE["oof"] is not None:
            p_oof = BLEND_STATE["oof"].reshape(-1)
            p_test= BLEND_STATE["test"].reshape(-1)
        else:
            if not LOADED["run_ids"]:
                print("Нет источника предсказаний (соберите бленд или загрузите run_id).")
                return
            rid = LOADED["run_ids"][0]
            p_oof = LOADED["oof"][rid].reshape(-1)
            p_test= LOADED["test"][rid].reshape(-1)
        cal_mode = CALIB_MODE.value
        cal_obj, apply_fn = fit_calibrator(cal_mode, y, p_oof)
        p_oof_cal = apply_fn(p_oof) if cal_obj else p_oof
        p_test_cal= apply_fn(p_test) if cal_obj else p_test
        tau, score = find_tau(y, p_oof_cal, THR_MODE.value)
        CALIB_STATE.update({"cal":cal_obj, "tau":tau, "oof_cal":p_oof_cal, "test_cal":p_test_cal})
        print("calibration:", cal_mode, "| τ:", tau, "|", THR_MODE.value, "score:", score)

display(w.HBox([CALIB_MODE, THR_MODE, BTN_CALIB], layout=ROW_LAYOUT), OUT_CALIB)
BTN_CALIB.on_click(on_calib)


In [ ]:
SUB_SRC = w.Dropdown(options=["blend"], value="blend", description="Источник")
SUB_TAG = w.Text(value=now_tag("submit"), description="SUB_TAG", layout=w.Layout(width="50%"))
BTN_SUB = w.Button(description="Собрать сабмит", button_style="success", layout=BTN_LAYOUT)
OUT_SUB = w.Output(layout={'border':'1px solid #ccc'})

def on_submit(_):
    OUT_SUB.clear_output(wait=True)
    with OUT_SUB:
        tag = RUN_TAG.value.strip()
        sd  = Path(SETS_DIR.value.strip() or f"artifacts/sets/{tag}")
        ids_te = try_read_parquet(sd/"ids_test.parquet")
        if ids_te is None:
            print("Нет ids_test.parquet")
            return
        id_col = ids_te.columns[0]
        # выбрать источник
        if SUB_SRC.value=="blend":
            if BLEND_STATE["test"] is None:
                print("Нет test предсказаний бленда.")
                return
            pred = BLEND_STATE["test"]
            # если есть калибровка/τ и от нас ожидается класс — применить при необходимости отдельно
        else:
            print("Сейчас поддержан только 'blend' как источник.")
            return
        # формируем CSV как score (id, target)
        df_sub = pd.DataFrame({id_col: ids_te[id_col].values, "target": pred.reshape(-1)})
        out_dir = ensure_dir(Path("artifacts")/"submits"/tag/SUB_TAG.value.strip())
        sub_path = out_dir/"submission.csv"
        df_sub.to_csv(sub_path, index=False)
        manifest = {
            "run_tag": tag,
            "source": SUB_SRC.value,
            "blend": {
                "type": BLEND_STATE["type"],
                "members": BLEND_STATE["members"],
                "weights": None if BLEND_STATE["weights"] is None else list(map(float, BLEND_STATE["weights"])),
                "metric_on_oof": BLEND_STATE["metric"]
            },
            "calibration": {
                "used": CALIB_STATE["cal"] is not None,
                "tau": None if CALIB_STATE["tau"] is None else float(CALIB_STATE["tau"]),
                "mode": CALIB_MODE.value if CALIB_MODE.value!="off" else "off"
            }
        }
        write_json(out_dir/"manifest.json", manifest)
        print("SUBMISSION →", sub_path.as_posix())

display(w.HBox([SUB_SRC, SUB_TAG, BTN_SUB], layout=ROW_LAYOUT), OUT_SUB)
BTN_SUB.on_click(on_submit)


In [ ]:
RUN_FOR_IMPORTANCE = w.Text(value="", description="run_id")
BTN_IMPORTANCE = w.Button(description="Показать важности", button_style="", layout=BTN_LAYOUT)
OUT_IMP = w.Output(layout={'border':'1px solid #ccc'})

def on_importance(_):
    OUT_IMP.clear_output(wait=True)
    with OUT_IMP:
        rid = RUN_FOR_IMPORTANCE.value.strip()
        if not rid:
            print("Укажи run_id (из сводной таблицы).")
            return
        p = Path("artifacts/models")/rid/"feature_importance.csv"
        if not p.exists():
            print("Нет feature_importance.csv")
            return
        df = pd.read_csv(p)
        df = df.sort_values("gain", ascending=False).head(20)
        display(df)
        plt.figure()
        plt.bar(df["feature"], df["gain"])
        plt.xticks(rotation=90); plt.title("Top-20 feature importance")
        plt.tight_layout(); plt.show()

display(w.HBox([RUN_FOR_IMPORTANCE, BTN_IMPORTANCE], layout=ROW_LAYOUT), OUT_IMP)
BTN_IMPORTANCE.on_click(on_importance)


In [ ]:
SLICE_COL = w.Text(value="", description="slice_col")
BTN_SLICE = w.Button(description="Посчитать по срезам", button_style="", layout=BTN_LAYOUT)
OUT_SLICE = w.Output(layout={'border':'1px solid #ccc'})

def on_slice(_):
    OUT_SLICE.clear_output(wait=True)
    with OUT_SLICE:
        tag = RUN_TAG.value.strip()
        sd  = Path(SETS_DIR.value.strip() or f"artifacts/sets/{tag}")
        Xd  = try_read_parquet(sd/"X_dense_train.parquet")
        if Xd is None:
            print("Нет dense train для срезов.")
            return
        col = SLICE_COL.value.strip()
        if not col or col not in Xd.columns:
            print("Укажи корректный столбец из dense набора.")
            return
        ydf = try_read_parquet(sd/"y_train.parquet")
        y   = ydf[[c for c in ydf.columns if c!=ydf.columns[0]][0]].to_numpy()
        task = TASK.value if TASK.value!="auto" else ("binary" if len(np.unique(y))<=2 else "multiclass")
        scorer = metric_fn(task, METRIC.value)
        # возьмём источник: бленд или первый загруженный
        if BLEND_STATE["oof"] is not None:
            p = BLEND_STATE["oof"]
        elif LOADED["run_ids"]:
            p = LOADED["oof"][LOADED["run_ids"][0]]
        else:
            print("Нет предсказаний для анализа.")
            return
        groups = pd.qcut(Xd[col], q=5, duplicates="drop") if pd.api.types.is_numeric_dtype(Xd[col]) else Xd[col].astype(str)
        df = pd.DataFrame({"g": groups, "y": y, "p": p if task!="multiclass" else p.max(1)})
        rows=[]
        for g, chunk in df.groupby("g"):
            rows.append({"slice": str(g), "metric": scorer(chunk["y"].values, chunk["p"].values)})
        out = pd.DataFrame(rows).sort_values("metric")
        display(out)

display(w.HBox([SLICE_COL, BTN_SLICE], layout=ROW_LAYOUT), OUT_SLICE)
BTN_SLICE.on_click(on_slice)


In [ ]:
BTN_SAVE_UI = w.Button(description="Сохранить UI", button_style="", layout=BTN_LAYOUT)
BTN_LOAD_UI = w.Button(description="Загрузить UI", button_style="", layout=BTN_LAYOUT)
OUT_UI = w.Output()

def on_save_ui(_):
    save_ui_state({
        "RUN_TAG": RUN_TAG.value, "SETS_DIR": SETS_DIR.value,
        "CANDS_MULTI": list(CANDS_MULTI.value),
        "TASK": TASK.value, "METRIC": METRIC.value, "DEVICE": DEVICE.value,
        "THREADS": THREADS.value, "SEED": SEED.value, "TIMEOUT": TIMEOUT.value,
        "RESUME": RESUME.value, "CALIB": CALIB.value, "SAVE_TEST": SAVE_TEST.value
    })
    display(badge("SAVED", "#0a0"))

def on_load_ui(_):
    st = load_ui_state()
    try:
        RUN_TAG.value = st.get("RUN_TAG", RUN_TAG.value)
        SETS_DIR.value= st.get("SETS_DIR", SETS_DIR.value)
        CANDS_MULTI.value = tuple(st.get("CANDS_MULTI", list(CANDS_MULTI.value)))
        TASK.value = st.get("TASK", TASK.value)
        METRIC.value = st.get("METRIC", METRIC.value)
        DEVICE.value = st.get("DEVICE", DEVICE.value)
        THREADS.value= st.get("THREADS", THREADS.value)
        SEED.value   = st.get("SEED", SEED.value)
        TIMEOUT.value= st.get("TIMEOUT", TIMEOUT.value)
        RESUME.value = st.get("RESUME", RESUME.value)
        CALIB.value  = st.get("CALIB", CALIB.value)
        SAVE_TEST.value = st.get("SAVE_TEST", SAVE_TEST.value)
        display(badge("LOADED", "#0a0"))
    except Exception as e:
        print("load err:", e)

BTN_SAVE_UI.on_click(on_save_ui)
BTN_LOAD_UI.on_click(on_load_ui)
display(w.HBox([BTN_SAVE_UI, BTN_LOAD_UI], layout=ROW_LAYOUT), OUT_UI)


### Чеклист перед сабмитом
- [ ] Формат CSV совпадает с регламентом (id, target)?
- [ ] Выбран лучший по OOF источник (или бленд)?
- [ ] Для AUC/PR-AUC сабмитим **скор/вероятность**, без порога.
- [ ] Если применяли калибровку/τ — это требовалось форматом?
- [ ] Файлы `blend_config.json` / `manifest.json` сохранены.